# Imports

In [1]:
# image manipulation
import cv2
import numpy as np
from PIL import Image
from skimage.transform import resize
from scipy.ndimage import zoom

# data format handling
import tifffile
import h5py

# utility, visualization, and path handling 
from tqdm import tqdm
from glob import glob
from matplotlib import pyplot as plt

# system configurations handling
from yacs.config import CfgNode as CN

# 3D tensor handling
import torch
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

import csv
from shutil import copyfile

# System configurations

In [2]:
_C = CN()

_C.EM_H5SEGGT = '/n/pfister_lab2/Lab/vcg_connectomics/EM/zebraFish/benchmark/seg_gt.h5'
_C.EXM_H5IMGS = '/n/pfister_lab2/Lab/zudilin/data/Expansion/image/dorsal_6_histeq.h5'
_C.ROOT = "/n/pfister_lab2/Lab/leander/cerberus/neuro_glancer/data"

# using the segmentation instead of the images
_C.EM_SEG = True

_C.EM_SCALE = tuple((2, 3, 3))
_C.EXM_SCALE = tuple((1, 0.25, 0.25))

_C.EM_IMG_SHAPE = (1450, 2000)
_C.EM_IMG_LEN = 397


_C.EM_D_BOUND_0 = 100
_C.EM_D_BOUND_1 = 305
_C.EM_H_BOUND_0 = 639
_C.EM_H_BOUND_1 = 810
_C.EM_W_BOUND_0 = 914
_C.EM_W_BOUND_1 = 1085

_C.TARGET_SIZE = (255,512,512)

def get_cfg_defaults():
  """Get a yacs CfgNode object with default values for my_project."""
  # Return a clone so that the defaults will not be altered
  # This is for the "local variable" use pattern
  return _C.clone()

cfg = get_cfg_defaults()
#cfg.merge_from_file("./configs/base_setup.yaml")
cfg.freeze()

# Util

In [3]:
def create_h5s(img_list, img_file_name, shape=None):
    """ Take a list of images and writes them to a h5 file
    
        Args:
            img_list: List of images that should be written to the h5-file
            img_file_name: Name for the h5-file
            shape: shape of the images
    """

    # open a h5-file for writing
    with h5py.File(img_file_name, "w") as img_patches:
        
        imgh5 = img_patches.create_dataset('main',shape=shape, dtype=int)
    
        # iterate over all images in the list
        for i, img in tqdm(enumerate(img_list)):
            # add the resized image to the dataset
            imgh5[i:i+1,:,:] = img

# Data

#### Patch the images

In [4]:
import os 
try:
    os.makedirs(cfg.ROOT, exist_ok=False)
except FileExistsError as e:
    print(e)

[Errno 17] File exists: '/n/pfister_lab2/Lab/leander/cerberus/neuro_glancer/data'


In [5]:
# open the source files for reading
with h5py.File(cfg.EM_H5SEGGT) as EM, \
        h5py.File(cfg.EXM_H5IMGS) as ExM:
        
    
    # retrive the GT segmentation
    em = np.asarray(EM.get(list(EM.keys())[0]))[cfg.EM_D_BOUND_0:cfg.EM_D_BOUND_1,
                                                cfg.EM_H_BOUND_0:cfg.EM_H_BOUND_1,
                                                cfg.EM_W_BOUND_0:cfg.EM_W_BOUND_1]
    # retrive the GT segmentation
    exm = np.asarray(ExM.get(list(ExM.keys())[0]))
    
    print(em.shape)
    print(cfg.EM_SCALE)
    em_final = np.zeros(cfg.TARGET_SIZE)
    exm_final = np.zeros(cfg.TARGET_SIZE)
    print(em_final.shape)
    print(exm_final.shape)
    
    for i, e in tqdm(enumerate(em)):      
        if cfg.EM_SEG:
            e = resize(e, cfg.TARGET_SIZE[1:], order=0, anti_aliasing=False, preserve_range=True)
        else:
            e = resize(e, cfg.TARGET_SIZE[1:], order=1, preserve_range=True, anti_aliasing=True)

        assert(e.shape == em_final.shape[1:]), f"the actual shape is {e.shape} "

        em_final[i, :, :] = e
        
    for i, e in tqdm(enumerate(exm)): 
        e = resize(e, cfg.TARGET_SIZE[1:], order=1, preserve_range=True, anti_aliasing=True)
        

        assert(e.shape == exm_final.shape[1:]), f"the actual shape is {e.shape}"

        exm_final[i, :, :] = e



    em_name = 'em_3D_'+'_'.join([ str(e) for e in cfg.EM_SCALE])+'.h5'
    exm_name = 'exm_3D_1_025_025.h5'
    em_path = os.path.join(cfg.ROOT, em_name)
    exm_path = os.path.join(cfg.ROOT, exm_name)
    
    print(em_path)
    print(exm_path)

    create_h5s(exm_final, exm_path, exm_final.shape)
    create_h5s(em_final, em_path, em_final.shape)

25it [00:00, 246.09it/s]

(205, 171, 171)
(2, 3, 3)
(255, 512, 512)
(255, 512, 512)


205it [00:00, 256.26it/s]
255it [00:34,  7.44it/s]
34it [00:00, 339.70it/s]

/n/pfister_lab2/Lab/leander/cerberus/neuro_glancer/data/em_3D_2_3_3.h5
/n/pfister_lab2/Lab/leander/cerberus/neuro_glancer/data/exm_3D_1_025_025.h5


255it [00:00, 425.18it/s]
255it [00:00, 390.52it/s]


In [6]:
# open the source files for reading
with h5py.File(cfg.EM_H5SEGGT) as EM, \
        h5py.File(cfg.EXM_H5IMGS) as ExM:
        
    
    # retrive the GT segmentation
    em = np.asarray(EM.get(list(EM.keys())[0]))[cfg.EM_D_BOUND_0:cfg.EM_D_BOUND_1,
                                                cfg.EM_H_BOUND_0:cfg.EM_H_BOUND_1,
                                                cfg.EM_W_BOUND_0:cfg.EM_W_BOUND_1]

    # retrive the GT segmentation
    exm = np.asarray(ExM.get(list(ExM.keys())[0]))
    
    
    print(em.shape)
    print(exm.shape)
    
    if cfg.EM_SEG:
        em_zoom = zoom(em, cfg.EM_SCALE, order=0) #resize(e, cfg.TARGET_SIZE[1:], order=0, anti_aliasing=False, preserve_range=True)
    else:
        em_zoom  = zoom(em, cfg.EM_SCALE, order=1) #resize(e, cfg.TARGET_SIZE[1:], order=1, preserve_range=True, anti_aliasing=True)

        assert(em_zoom .shape == tuple([ int(e*s) for e,s in zip(em.shape,cfg.EM_SCALE)])), f"the actual shape is {em.shape}"

        
    exm_zoom = zoom(exm, cfg.EM_SCALE, order=1) #resize(e, cfg.TARGET_SIZE[1:], order=1, preserve_range=True, anti_aliasing=True)
        
    assert(exm_zoom.shape == tuple([ int(e*s) for e,s in zip(exm.shape,cfg.EXM_SCALE)])), f"the actual shape is {e.shape}"




    em_name = 'em_3D_'+_.join(cfg.EM_SCALE)+'.h5'
    exm_name = 'seg_3D_'+_.join(cfg.EXM_SCALE)+'.h5'
    em_path = os.path.join(cfg.ROOT, em_name)
    exm_path = os.path.join(cfg.ROOT, exm_name)
    
    print(em_path)
    print(exm_path)

    create_h5s(exm_zoom, exm_path, exm_final.shape)
    create_h5s(em_zoom, em_path, em_final.shape)

(205, 171, 171)
(255, 2048, 2048)


AssertionError: the actual shape is (512, 512)